# Importação dos dados e bibliotecas

In [ ]:
!python -m pip install nbformat notebook pandas plotly seaborn numpy scikit-learn matplotlib

## Importação das bibliotecas

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

## Importação dos arquivos csv

In [ ]:
data_2020_2021 = pd.read_csv("./dados/base_inteli 2020_2021.csv", sep=";" , encoding='utf8')
df_1 = pd.DataFrame(data_2020_2021)


In [ ]:
data_2022_2023 = pd.read_csv("./dados/base_inteli_2022_2023.csv", sep =";", encoding='utf8')
df_2 = pd.DataFrame(data_2022_2023)

Concatenação dos databases

In [ ]:
df = pd.concat([df_1,df_2])

# Exploração dos dados

## Identificação das colunas numéricas e categóricas


Identificação dos tipos de dados de cada coluna pelo padrão do arquivo:

In [ ]:
data_types = df.dtypes
data_types

Foram realizadas a descrição e a categorização das colunas para melhor entendimento dos dados e auxiliar posteriormente no pré-processamento dessas colunas:

| Nome de Origem | Descrição | Tipo | Identificação | Sub-identificação |
|----------------|-----------|------|----------------|-------------------|
| date | Data da compra | string | Categórico | Ordinal |
| weekday_name | Dia que a compra foi realizada na semana (Ex. Thursday) | string | Categórico | Ordinal |
| sku | Código único do produto vendido | string | Categórico | Nominal |
| unit_price | Preço do produto | float | Numérico | Contínuo |
| mobly_item | Se o produto é entregue e vendido pela mobly.  Pode ser not mobly item ou mobly_item | bool | Categórico | Nominal |
| shipment_type | A forma que o produto é entregue | string | Categórico | Nominal |
| anchor_category | É tipo/categoria macro do produto | string | Categórico | Nominal |
| product_department | Departamento do produto | string | Categórico | Nominal |
| product_category | Ambientação que o produto será usado/categoria micro | string | Categórico | Nominal |
| origin_country | Origem do produto (Ex. nacional ou importado) | string | Categórico | Nominal |
| process_costing | Se a mobly fornece ou não matéria prima para a produção do produto | bool | Categórico | Nominal |
| sku_color | Cor do produto | string | Categórico | Nominal |
| sku_height | Altura do produto | float | Numérico | Contínuo |
| sku_width | Largura do produto | float | Numérico | Contínuo |
| sku_length | Comprimento do produto | float | Numérico | Contínuo |
| sku_weight | Peso do produto | float | Numérico | Contínuo |
| price_status | Comparação com outros produtos no google shopping  para saber se está ganhando, perdendo ou empatando. | bool | Categórico | Nominal |
| winning_price | Preço do produto que está ganhando no google shopping | float | Numérico | Contínuo |
| revenue | Receita daquele produto | float | Numérico | Contínuo |
| items_sold | Número de produtos vendidos | int | Numérico | Discreto |
| avg_website_visits_last_week | Média de visitação do site | float | Numérico | Contínuo |
| supplier_delivery_time | Tempo de entrega do fornecedor | int | Numérico | Discreto |
| stock_qty | Quantidade de itens no estoque | int | Numérico | Discreto |
| flag_bundle | Diz se o item é vendido separado ou em conjunto | bool | Categórico | Nominal |
| revenue_bundle | Receita do bundle(conjunto de proputos) | float | Numérico | Contínuo |
| items_sold_bundle | Número de itens que são vendidos em conjunto | int | Numérico | Discreto' |


## Estatística descritiva das colunas

Descrição estatística das colunas numéricas:

In [ ]:
df.describe()

## Gráficos

O gráfico de barras abaixo ilustra a relação entre a quantidade de itens vendidos e o seu status frente aos concorrentes, ou seja, se ele está sendo vendido mais caro (perdendo), mais barato (ganhando), ao mesmo preço (empatando) ou se é vendido apenas na Mobly (único disponível). O gráfico abaixo foi criado para entender a relação entre a quantidade de vendas de um produto e o seu preço no mercado em relação à concorrência. A partir da criação desse gráfico, foi possível perceber que a categoria “Sem registro” compõe creca de 80% dos itens vendidos, o que pode impossibilitar uma análise mais assertiva desse gráfico.

In [ ]:
# somando a quantidadxe de items vendidos por grupo de price_status
grouped_df = df.groupby('price_status').agg({'items_sold': 'sum'}).reset_index()

fig_1_df = pd.DataFrame(grouped_df[["price_status","items_sold"]])
fig_1_df = fig_1_df.rename(columns={"price_status":"Status do Produto","items_sold":"Itens Vendidos (milhões)"})

plt.figure(figsize=(12, 6))
fig_1 = sns.barplot(data=fig_1_df ,x='Status do Produto', y='Itens Vendidos (milhões)',palette=("yellow","green","red","gray","blue")).set_title("Gráfico de Barras de Vendas em Relação à Concorrência")

O gráfico abaixo relaciona a quantidade de itens vendidos por categoria e por cor, ou seja, evidencia a partir do heatmap quais os itens mais vendidos baseado nos tons mais próximos de azul escuro, mostrando em cada quadrado a quantidade de vendas, e as linhas de cores estão ordenadas pelas cores mais vendidas.

In [ ]:
vendidos = df[df["items_sold"]>0]

fig_2 = vendidos.pivot_table(index='sku_color', columns='anchor_category', aggfunc='size', fill_value=0)

fig_2['Soma'] = fig_2.sum(axis=1)
fig_2 = fig_2.sort_values(by='Soma', ascending=False)
fig_2.drop('Soma', axis=1, inplace=True)

plt.figure(figsize=(18, 15))
fig_2 = sns.heatmap(fig_2, annot=True, fmt='d', cmap='YlGnBu', annot_kws = {'size': 8} )
plt.title('Quantidade de Itens Vendidos por Cor e Categoria')
plt.show()


O gráfico de linha apresenta o volume total de itens vendidos para cada dia da semana, de segunda-feira a domingo. A altura de cada ponto na linha indica o número de itens vendidos em um dia específico, permitindo uma visualização rápida da distribuição de vendas ao longo da semana.

In [ ]:
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig_3_df = pd.DataFrame(df.groupby('weekday_name')['items_sold'].sum())

fig_3_df = fig_3_df.reindex(days_order)

fig_3 = fig_3_df.plot()
plt.ylim(0,400000)
fig_3.set(ylim=(0,400000))
fig_3.set_title('Gráfico de vendas totais por dia da semana')
plt.show()

# Hipóteses

## Hipótese 1: A cor de um produto influencia na sua venda, uma vez que as cores do produto mais vendidas são preto, branco e cinza, respectivamente.

> Motivação: No desenvolvimento do modelo preditivo de vendas, foi percebido empiricamente que a cor de um produto poderia influenciar na sua compra e que as cores mais vendidas diferenciariam dependendo da categoria do produto. Para entender melhor essa ideia, foi construido um heatmap(mapa de calor) que mostra a quantidade de vendas por cor e por categoria de produto, de modo que a tonalidade da cor de cada quadrado indica sua quantidade de vendas e há uma matriz cor x categoria para analisar. Portanto, ao explorar esses atributos, será possível entender se há uma tendência de compra influenciado pela cor. 

In [ ]:
fig_2 = vendidos.pivot_table(index='sku_color', columns='anchor_category', aggfunc='size', fill_value=0)

fig_2['Soma'] = fig_2.sum(axis=1)
fig_2 = fig_2.sort_values(by='Soma', ascending=False)
fig_2.drop('Soma', axis=1, inplace=True)

plt.figure(figsize=(18, 15))
fig_2 = sns.heatmap(fig_2, annot=True, fmt='d', cmap='YlGnBu', annot_kws = {'size': 8} )
fig_2.set_title('Quantidade de Itens Vendidos por Cor e Categoria')
plt.show()

> Justificativa: Ao construir o gráfico acima, foi possível observar uma maior quantidade de compra de itens de cor neutra, como preto, branco e cinza. Além disso, foi notada uma tendência de cores das categorias mais vendidas: para sofás, as cores mais vendidas são cinza, bege e cru, enquanto para office chairs as cores com maior venda são preto, branco e marrom. Isso evidencia que a ocorrência de cores depende da categoria, mas permaecem no padrão de cores neutras. 

## Hipótese 2: A média de visitação no site se mantém constante durante toda a semana.

> Motivação: O comportamento constante das visitações no site pode ser relacionado à média de vendas, permitindo analisar se o comportamento das visitações segue o mesmo padrão das vendas por dia da semana e quantas visitações são convertidas em vendas. Essa análise pode contribuir para ajustar campanhas de marketing e operações logísticas, visando aumentar o número de visitações convertidas em vendas.

In [ ]:
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig_4_df = pd.DataFrame(df.groupby('weekday_name')['avg_website_visits_last_week'].sum())

fig_4_df = fig_4_df.reindex(days_order)

fig_4 = fig_4_df.plot()
plt.ylim(10000000,15000000)
fig_4.set(ylim=(10000000,15000000))
fig_4.set_title('Gráfico da média de visitações por dia da semana')
plt.show()

> Justificativa: A análise do gráfico de visitações médias no site da Mobly por dia da semana indica que a média de visitações é constante durante toda a semana, mostrando pouca variação diária. Isso sugere que os clientes da Mobly acessam a plataforma de forma consistente, ou seja, esse comportamento regular indica uma base de clientes menos influenciada por fatores semanais, o que pode auxiliar no aprimoramento do modelo preditivo de vendas e na estratégia de marketing da empresas..

## Hipótese 3: As vendas da Mobly são consistentes ao longo dos dias da semana.

> Motivação: O comportamento consistente das vendas diárias pode simplificar e tornar mais eficaz o modelo preditivo de vendas. Sabendo que o e-commerce tem transformado os padrões de compra, entender se as vendas variam ou permanecem uniformes ao longo da semana é crucial, não apenas para o modelo, mas também para ajustar campanhas de marketing e operações logísticas.

In [ ]:
fig_3 = fig_3_df.plot()
plt.ylim(0,400000)
fig_3.set(ylim=(0,400000))
fig_3.set_title('Gráfico de vendas totais por dia da semana')
plt.show()

> Justificativa: A análise do gráfico de vendas da Mobly por dia da semana indica que os volumes de vendas são semelhantes, mostrando pouca variação diária. Isso sugere que os clientes da Mobly compram de forma consistente, independentemente do dia da semana. Esse comportamento regular indica uma base de clientes leais e menos influenciada por fatores semanais, o que pode auxiliar no aprimoramento do modelo preditivo de vendas.